In [ ]:
# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
import io
import base64
from io import BytesIO
from PIL import Image
from PIL import ImageOps
import numpy as np

# For running inference on the TF-Hub module.
import tensorflow as tf
import tensorflow_hub as hub

# For measuring the inference time.
import time

from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont


In [ ]:
# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

# Download Example Images:

In [ ]:
def display_image(image):
    fig = plt.figure(figsize=(20, 15))
    plt.grid(False)
    plt.imshow(image)

def display_image_from_base64(image_data):
    imgdata = base64.b64decode(image_data)
    image = Image.open(BytesIO(imgdata))
    display_image(image)
    
    
def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
    _, filename = tempfile.mkstemp(suffix=".jpg")
    response = urlopen(url)
    image_data = response.read()
    image_data = BytesIO(image_data)
    pil_image = Image.open(image_data)
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    pil_image_rgb = pil_image.convert("RGB")
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    print("Image downloaded to %s." % filename)
    if display:
        display_image(pil_image)
    return filename

In [ ]:
# By Heiko Gorski, Source: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
#image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg"  #@param

#downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)

image_urls = [
  # Source: https://commons.wikimedia.org/wiki/File:The_Coleoptera_of_the_British_islands_(Plate_125)_(8592917784).jpg
  "https://upload.wikimedia.org/wikipedia/commons/1/1b/The_Coleoptera_of_the_British_islands_%28Plate_125%29_%288592917784%29.jpg",
  # By Américo Toledano, Source: https://commons.wikimedia.org/wiki/File:Biblioteca_Maim%C3%B3nides,_Campus_Universitario_de_Rabanales_007.jpg
  "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg/1024px-Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg",
  # Source: https://commons.wikimedia.org/wiki/File:The_smaller_British_birds_(8053836633).jpg
  "https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg",
]

img_list = []
for image_url in image_urls:
    img_list.append(download_and_resize_image(image_url, 1280, 856, True))

# Transform Images to String:

In [ ]:
images = []
for item in img_list:
    with open(item, "rb") as img_file:
        images.append(base64.b64encode(img_file.read()))
        
print(images[0])

In [ ]:
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
    """Adds a bounding box to an image."""
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

    # If the total height of the display strings added to the top of the bounding
    # box exceeds the top of the image, stack the strings below the bounding box
    # instead of above.
    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
    # Each display_str has a top and bottom margin of 0.05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = top + total_display_str_height
    # Reverse list and print from bottom to top.
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
        text_bottom -= text_height - 2 * margin



def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
    """Overlay labeled boxes on an image with formatted scores and label names."""
    colors = list(ImageColor.colormap.values())

    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
    except IOError:
        print("Font not found, using default font.")
        font = ImageFont.load_default()

    for i in range(min(boxes.shape[0], max_boxes)):
        if scores[i] >= min_score:
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                         int(100 * scores[i]))
            color = colors[hash(class_names[i]) % len(colors)]
            image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
            draw_bounding_box_on_image(
              image_pil,
              ymin,
              xmin,
              ymax,
              xmax,
              color,
              font,
              display_str_list=[display_str])
            np.copyto(image, np.array(image_pil))
    return image

In [ ]:
def image_base64_to_input_format(image_data):
    imgdata = base64.b64decode(image_data)
    image = Image.open(BytesIO(imgdata))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((1, im_height, im_width, 3)).astype(np.uint8)

def input_format_to_image_base64(input_image):
    image = Image.fromarray( np.asarray(input_image))
    with BytesIO() as buffer:
        image.save(buffer, format="PNG")
        image_data = buffer.getvalue()
    image_base64 = base64.b64encode(image_data).decode('utf-8')
    return image_base64

# Application:

Objectdetection: the application takes as input an image and returns the objects that have been detected on the image. It should also work for multiple images. Images are sent to the application via Post request as an array of base64 encoded Strings. Results should be returned as JSON Response.

# Version 1:

In [ ]:
#module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
#module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

def detection_loop(images):
    bounding_boxes = []
    inf_times = []
    for image in images:
        # decode image string:
        image_tensor = image_base64_to_input_format(image)
        #img = np.array(Image.open(io.BytesIO(base64.b64decode(image))))
        #display_image(img)

        #converted_img  = tf.image.convert_image_dtype(decoded_tensor, tf.float32)[tf.newaxis, ...]
        start_time = time.time()
        result = detector(image_tensor)
        end_time = time.time()

        #result = {key:value.numpy() for key,value in result.items()}
        colors = np.array([[1.0, 0.0, 0.0], [0.0, 0.0, 1.0]])
        boxes = result["detection_boxes"][0][:20]
        print( boxes)
        expanded =  np.expand_dims(boxes, axis=0)
        res_image = tf.image.draw_bounding_boxes(image_tensor, expanded, colors)
        #display_image()
      
        res_image = np.squeeze(res_image)
        res_image = res_image.astype(np.uint8)
# Display the image using Matplotlib
        plt.imshow(res_image)
        plt.axis('off')  # Turn off axis labels
        plt.show()
        # display_image = input_format_to_image_base64(res_image)
        # display_image_from_base64(display_image)
        # print("Found %d objects." % len(result["num_detections"]))
        # print("Inference time: ", end_time-start_time)
        # print(result["detection_boxes"],)

        bounding_boxes.append(result["detection_boxes"])
        inf_times.append(end_time-start_time)
        print("Done!")
"""
        data = {"status": 200,
              "bounding_boxes": bounding_boxes,
              "inf_time": inf_times,
              "avg_inf_time": str(np.mean(inf_times)),
              "upload_time": [],
              "avg_upload_time": ""}
"""
    #return make_response(jsonify(data), 200)
    #return(data)


result = detection_loop(images)



In [ ]:
result["inf_time"]

# Version 2 (with load images):

In [ ]:
def detection_loop(filename_image):
  # input is a list of filenames to images

  # load object detection module:
  module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
  detector = hub.load(module_handle).signatures['default']

  bounding_boxes = []
  inf_times = []
  upload_times = []
  for filename in filename_image:

    # load image:
    start_time = time.time()
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    end_time = time.time()
    upload_times.append(end_time-start_time)

    # detect objects on image:
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    start_time = time.time()
    result = detector(converted_img)
    end_time = time.time()

    result = {key:value.numpy() for key,value in result.items()}

    print("Found %d objects." % len(result["detection_scores"]))
    print("Inference time: ", end_time-start_time)

    bounding_boxes.append(result["detection_boxes"])
    inf_times.append(end_time-start_time)
  print("Detection Done!")

  data = {"status": 200,
          "bounding_boxes": bounding_boxes,
          "inf_time": inf_times,
          "avg_inf_time": str(np.mean(inf_times)),
          "upload_time": upload_times,
          "avg_upload_time": str(np.mean(upload_times))}

  return make_response(jsonify(data), 200)